In [1]:
from keras.models import Model
from keras.layers import LSTM, Input
from keras.callbacks import LearningRateScheduler
from keras.utils.np_utils import to_categorical
from PointerLSTM import PointerLSTM
import pickle
import tsp_data as tsp
import sort_data as sort
import numpy as np

Using TensorFlow backend.


In [2]:
seq_len = 5
BATCH = 3
hidden_size = 128
nb_epochs =  10
learning_rate = 0.3

In [3]:
t = tsp.Tsp()
X, Y = t.next_batch(BATCH, seq_len)
x_test, y_test = t.next_batch(2)
YY = []
for y in Y:
    YY.append(to_categorical(y))
YY = np.asarray(YY)

preparing dataset... 0/3
preparing dataset... 0/2


In [4]:
main_input = Input(shape=(seq_len, 2), name='main_input')

In [5]:
encoder = LSTM(output_dim = hidden_size, return_sequences = True, name="encoder")(main_input)

/Users/james/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=128, return_sequences=True, name="encoder")`
  if __name__ == '__main__':


In [6]:
decoder = PointerLSTM(hidden_size, output_dim=hidden_size, name="decoder")(encoder)

PointerLSTM.py:12: UserWarning: Update your `PointerLSTM` call to the Keras 2 API: `PointerLSTM(units=128, name="decoder")`
  super(PointerLSTM, self).__init__(*args, **kwargs)


Tensor("encoder/transpose_1:0", shape=(?, ?, 128), dtype=float32)


In [7]:
model = Model(inputs=main_input, outputs=decoder)

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 5, 2)              0         
_________________________________________________________________
encoder (LSTM)               (None, 5, 128)            67072     
_________________________________________________________________
decoder (PointerLSTM)        (None, 5, 5)              131845    
Total params: 198,917
Trainable params: 198,917
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(optimizer='adadelta',
              loss='categorical_crossentropy',
              metrics=['accuracy'])